### Import the 100k Movielens Dataset called 'ratings_2.csv'

In this notebook, I ran implicit on the real Movielens 100k data in order to get the metric results. I did this so I could compare the real results to the results from the synthetic data.

In [1]:
import pandas as pd
ratings_df = pd.read_csv('ratings_2.csv')
del ratings_df['timestamp']
ratings_df.tail()

C:\Users\micha\Anaconda3\envs\michael\lib\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\micha\Anaconda3\envs\michael\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
C:\Users\micha\Anaconda3\envs\michael\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


,userId,movieId,rating
99995,880,476,3
99996,716,204,5
99997,276,1090,1
99998,13,225,2
99999,12,203,3


#### Create a binary rating column in order to be able to run implicit on the data

In [2]:
ratings_df['rating2'] = ratings_df['rating'].apply(lambda x: '0' if x <= 3 else '1')
ratings_df.tail()

,userId,movieId,rating,rating2
99995,880,476,3,0
99996,716,204,5,1
99997,276,1090,1,0
99998,13,225,2,0
99999,12,203,3,0


In [3]:
del ratings_df['rating']
ratings_df['rating'] = ratings_df['rating2']
del ratings_df['rating2']
ratings_df.tail()

,userId,movieId,rating
99995,880,476,0
99996,716,204,1
99997,276,1090,0
99998,13,225,0
99999,12,203,0


In [4]:
import sys
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random

from sklearn.preprocessing import MinMaxScaler

import implicit
from implicit.evaluation import ranking_metrics_at_k, train_test_split

In [5]:
sparse_item_user = sparse.coo_matrix((ratings_df['rating'].astype(float), (ratings_df['movieId'], ratings_df['userId'])))
sparse_user_item = sparse.coo_matrix((ratings_df['rating'].astype(float), (ratings_df['userId'], ratings_df['movieId'])))

### Precision

In [6]:
from implicit.evaluation import precision_at_k, train_test_split
from implicit.als import AlternatingLeastSquares

users, movies, ratings = ratings_df
train, test = train_test_split(sparse_item_user)

model = AlternatingLeastSquares(factors=100, regularization=5, iterations=15)
model.fit(train)

p = precision_at_k(model, train.T.tocsr(), test.T.tocsr(), K=10, num_threads=4)

In [7]:
p

0.3126079447322971

### nDCG

In [8]:
from implicit.evaluation import ndcg_at_k, train_test_split
from implicit.als import AlternatingLeastSquares

users, movies, ratings = ratings_df
train, test = train_test_split(sparse_item_user)

model = AlternatingLeastSquares(factors=100, regularization=5, iterations=15)
model.fit(train)

ndcg = ndcg_at_k(model, train.T.tocsr(), test.T.tocsr(), K=10, num_threads=4)

In [9]:
ndcg

0.3032896278817956

### MAP

In [10]:
from implicit.evaluation import mean_average_precision_at_k, train_test_split
from implicit.als import AlternatingLeastSquares

users, movies, ratings = ratings_df
train, test = train_test_split(sparse_item_user)

model = AlternatingLeastSquares(factors=100, regularization=5, iterations=15)
model.fit(train)

map = mean_average_precision_at_k(model, train.T.tocsr(), test.T.tocsr(), K=10, num_threads=4)

In [11]:
map

0.18541339799934808